In [1]:
!pip install iterative-stratification
!pip install transforms3d

     |████████████████████████████████| 71kB 5.1MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-cp36-none-any.whl size=59374 sha256=231b58ecb69b56cf62243ebcf5e1d2320a4b4763e4ae26e77be8c1da17568d3c
  Stored in directory: /root/.cache/pip/wheels/3d/3c/84/28d36677f3c760c048bd02b5a547ea0c4027770cc9cdb9af1e
Successfully built transforms3d


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/운동동작분류AI경진대회')

In [4]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

import os, gc, random, datetime
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load
from time import time
import scipy as sp
import scipy.fftpack

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold

print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.4.1


# Preprocessing

In [5]:
# 데이터 불러오기

path = './data/'
train = pd.read_csv(path + 'train_features.csv')
train_label = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')

In [6]:
# Pre-Processing Effect on the Accuracy of Event-Based Activity Segmentation and Classification through Inertial Sensors 
# https://www.researchgate.net/publication/281836367_Pre-Processing_Effect_on_the_Accuracy_of_Event-Based_Activity_Segmentation_and_Classification_through_Inertial_Sensors

train['acc_t'] = train.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 + x['acc_z'] **2)**(1/2), axis=1)
test['acc_t'] = test.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 + x['acc_z'] **2)**(1/2), axis=1)

train['gy_t'] = train.apply(lambda x : (x['gy_x']**2 + x['gy_y'] **2 + x['gy_z'] **2)**(1/2), axis=1)
test['gy_t'] = test.apply(lambda x : (x['gy_x']**2 + x['gy_y'] **2 + x['gy_z'] **2)**(1/2), axis=1)

# # SVM selected features
# train['mean'] = train[['acc_x','acc_y']].mean(axis=1)
# train['median'] = train[['acc_y', 'gy_z', 'gy_t']].median(axis=1)
# train['standard_deviation'] = train[['acc_x', 'acc_y']].std(axis=1)
# train['interquartile'] = train.quantile(.75, axis=1) - train.quantile(.25, axis=1)

# test['mean'] = test[['acc_x','acc_y']].mean(axis=1)
# test['median'] = test[['acc_y', 'gy_z', 'gy_t']].median(axis=1)
# test['standard_deviation'] = test[['acc_x', 'acc_y']].std(axis=1)
# test['interquartile'] = test.quantile(.75, axis=1) - test.quantile(.25, axis=1)


In [7]:
feature_acc = ['acc_x', 'acc_y', 'acc_z', 'acc_t']
feature_gy = ['gy_x', 'gy_y', 'gy_z', 'gy_t']

train['acc_sum'] = train[feature_acc].sum(axis=1)
train['acc_mean'] = train[feature_acc].mean(axis=1)
train['acc_median'] = train[feature_acc].median(axis=1)
train['acc_min'] = train[feature_acc].min(axis=1)
train['acc_max'] = train[feature_acc].max(axis=1)
train['acc_std'] = train[feature_acc].std(axis=1)
train['acc_var'] = train[feature_acc].var(axis=1)
train['acc_skew'] = train[feature_acc].skew(axis=1)

train['gy_sum'] = train[feature_gy].sum(axis=1)
train['gy_mean'] = train[feature_gy].mean(axis=1)
train['gy_median'] = train[feature_gy].median(axis=1)
train['gy_min'] = train[feature_gy].min(axis=1)
train['gy_max'] = train[feature_gy].max(axis=1)
train['gy_std'] = train[feature_gy].std(axis=1)
train['gy_var'] = train[feature_gy].var(axis=1)
train['gy_skew'] = train[feature_gy].skew(axis=1)

test['acc_sum'] = test[feature_acc].sum(axis=1)
test['acc_mean'] = test[feature_acc].mean(axis=1)
test['acc_median'] = test[feature_acc].median(axis=1)
test['acc_min'] = test[feature_acc].min(axis=1)
test['acc_max'] = test[feature_acc].max(axis=1)
test['acc_std'] = test[feature_acc].std(axis=1)
test['acc_var'] = test[feature_acc].var(axis=1)
test['acc_skew'] = test[feature_acc].skew(axis=1)

test['gy_sum'] = test[feature_gy].sum(axis=1)
test['gy_mean'] = test[feature_gy].mean(axis=1)
test['gy_median'] = test[feature_gy].median(axis=1)
test['gy_min'] = test[feature_gy].min(axis=1)
test['gy_max'] = test[feature_gy].max(axis=1)
test['gy_std'] = test[feature_gy].std(axis=1)
test['gy_var'] = test[feature_gy].var(axis=1)
test['gy_skew'] = test[feature_gy].skew(axis=1)

In [8]:
x = np.array(train.iloc[:,2:]).reshape(-1, 600, 24)
y = tf.keras.utils.to_categorical(train_label['label'])
test = np.array(test.iloc[:,2:]).reshape(-1, 600, 24)

## Rotation

## Data augmentation for time-series data

https://github.com/terryum/Data-Augmentation-For-Wearable-Sensor-Data/blob/master/Example_DataAugmentation_TimeseriesData.ipynb

In [9]:
# # 26번을 제외한 id 리스트
# feature = list(train_label[train_label['label'] != 26]['id'])

In [10]:
# # train 데이터에서 26번을 삭제시킨다.
# temp = []
# for n in tqdm(range(train.shape[0])):
#     if train['id'][n] in feature:
#         temp.append(train.iloc[n])

In [11]:
# # 26번을 삭제시킨 데이터프레임
# without = pd.DataFrame(data=np.array(temp), columns=train.columns)
# without = without.astype({'id':int, 'time':int})

In [12]:
# without_train = np.array(without.iloc[:,2:]).reshape(-1, 600, 24)
# without_label = train_label[train_label['label'] != 26]['label']

In [13]:
# def aug(data, shift):
#     shift_data = np.roll(data, shift, axis=1)
#     return shift_data

# # 데이터 증강
# shift_data = []
# shift_label = []
# for n in tqdm(range(1, 10)):
#     shifted = aug(without_train, n*60)
#     shift_data.append(shifted)
#     shift_label.append(without_label)

# shift_data = np.array(shift_data).reshape(-1,600,24)
# shift_label = np.array(shift_label).reshape(-1,1)
# shift_categorical = tf.keras.utils.to_categorical(shift_label)

In [14]:
# # 원본 데이터와 증강 데이터 합치기
# concat_train = np.concatenate((x, shift_data), axis=0)
# concat_label = np.concatenate((y, shift_categorical), axis=0)
# print(concat_train.shape)
# print(concat_label.shape)

In [15]:
# del x, y, feature_acc, feature_gy, feature, without, without_train, without_label, train

# Training

Base Transformer structure from https://www.tensorflow.org/tutorials/text/transformer, modified with Swish activation function.

In [16]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b = True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads):
        
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm = [0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
        return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation = 'relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, dff, rate = 0.1):
        
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training = training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class TransformerEncoder(tf.keras.layers.Layer):
    
    def __init__(self, num_layers, d_model, num_heads, dff, 
                 maximum_position_encoding, rate = 0.1):
        
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.maximum_position_encoding = maximum_position_encoding
        self.rate = rate

#         self.pos_encoding = positional_encoding(self.maximum_position_encoding, 
#                                                 self.d_model)
#         self.embedding = tf.keras.layers.Dense(self.d_model)
        self.pos_emb = tf.keras.layers.Embedding(input_dim = self.maximum_position_encoding, 
                                                 output_dim = self.d_model)

        self.enc_layers = [EncoderLayer(self.d_model, self.num_heads, self.dff, self.rate) 
                           for _ in range(self.num_layers)]

        self.dropout = tf.keras.layers.Dropout(self.rate)
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'maximum_position_encoding': self.maximum_position_encoding,
            'dropout': self.dropout,
        })
        return config

    def call(self, x, training, mask = None):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
#         x += self.pos_encoding[:, :seq_len, :]
#         x = self.embedding(x)
        positions = tf.range(start = 0, limit = seq_len, delta = 1)
        x += self.pos_emb(positions)

        x = self.dropout(x, training = training)

        for i in range(self.num_layers):

            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [17]:
def create_transformer_model(num_columns, num_labels, num_layers, d_model, num_heads, dff, window_size, dropout_rate, weight_decay, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (window_size, num_columns))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dense(d_model)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.SpatialDropout1D(dropout_rate)(x)
    x = TransformerEncoder(num_layers, d_model, num_heads, dff, window_size, dropout_rate)(x)
    out = tf.keras.layers.Dense(num_labels, activation = 'softmax')(x[:, -1, :])
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['AUC'])
    
    return model

In [18]:
batch_size = 32
num_layers = 1
d_model = 128
num_heads = 1
dff = 128
window_size = 600
dropout_rate = 0.15
weight_decay = 0
label_smoothing = 1e-2
learning_rate = 1e-3
verbose = 1

In [19]:
# 모델 1번: Transformer

def build_transformer(split_num, train, target, test, rnd):
    start_time_fold = time()
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))

    ckp_path = 'transformer.hdf5'

    rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 3, verbose = verbose, min_delta = 1e-4, mode = 'min')
    ckp = ModelCheckpoint(ckp_path, monitor = 'val_loss', verbose = 0, save_best_only = True, save_weights_only = True, mode = 'min')
    es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 4, mode = 'min', baseline = None, restore_best_weights = True, verbose = 0)

    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in mskf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        #가벼운 모델 생성
        model = create_transformer_model(train.shape[2], 61, num_layers, d_model, num_heads, dff, window_size, dropout_rate, weight_decay, label_smoothing, learning_rate)

        model.fit(X, y, epochs = 30,
                  validation_data = (valid_x, valid_y),
                  batch_size = batch_size,
                  callbacks = [ckp, es],
                  verbose = verbose)
        
        # save feat
        model.load_weights(ckp_path)
        train_pred[val_idx] = model.predict(valid_x)
        test_pred += model.predict(test)/split_num
        
        # release
        del model
        gc.collect()
        print('  ==============================================================================================  ')

    return train_pred, test_pred

# transformer_pred = build_transformer(4, concat_train, concat_label, test, 1)
transformer_train, transformer_test = build_transformer(4, x, y, test, 1)

Epoch 1/30
79/79 [==============================] - 7s 40ms/step - loss: 3.0114 - auc: 0.7792 - val_loss: 2.2188 - val_auc: 0.9057
Epoch 2/30
79/79 [==============================] - 2s 30ms/step - loss: 2.1376 - auc: 0.9140 - val_loss: 1.7523 - val_auc: 0.9511
Epoch 3/30
79/79 [==============================] - 2s 30ms/step - loss: 1.8695 - auc: 0.9456 - val_loss: 1.6468 - val_auc: 0.9572
Epoch 4/30
79/79 [==============================] - 2s 30ms/step - loss: 1.7106 - auc: 0.9529 - val_loss: 1.5170 - val_auc: 0.9591
Epoch 5/30
79/79 [==============================] - 2s 30ms/step - loss: 1.5485 - auc: 0.9607 - val_loss: 1.3893 - val_auc: 0.9700
Epoch 6/30
79/79 [==============================] - 2s 31ms/step - loss: 1.5020 - auc: 0.9652 - val_loss: 1.3530 - val_auc: 0.9676
Epoch 7/30
79/79 [==============================] - 2s 31ms/step - loss: 1.4829 - auc: 0.9615 - val_loss: 1.3489 - val_auc: 0.9669
Epoch 8/30
79/79 [==============================] - 2s 31ms/step - loss: 1.3572 - a

In [20]:
# sample_submssion = pd.read_csv(path + 'sample_submission.csv')
# sample_submssion.iloc[:,1:] = transformer_pred
# sample_submssion.to_csv("transformer_30epoch.csv", index = False)
# sample_submssion

In [26]:
!pip install catboost

     |████████████████████████████████| 65.8MB 47kB/s 


In [27]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import random
import gc
from tqdm import tqdm
from glob import glob

import xgboost as xgb
import catboost as cb
import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import log_loss
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import skew, kurtosis
from sklearn.manifold import TSNE
import missingno

In [28]:
# 데이터 불러오기

path = './data/'
train = pd.read_csv(path + 'train_features.csv')
train_label = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [29]:
# Pre-Processing Effect on the Accuracy of Event-Based Activity Segmentation and Classification through Inertial Sensors 
# https://www.researchgate.net/publication/281836367_Pre-Processing_Effect_on_the_Accuracy_of_Event-Based_Activity_Segmentation_and_Classification_through_Inertial_Sensors

train['acc_t']  = train.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] ** 2 )**(1/3), axis=1)
test['acc_t']  = test.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] ** 2 )**(1/3), axis=1)

train['gy_t']  = train.apply(lambda x : (x['gy_x']**2 + x['gy_y'] **2 +  x['gy_z'] ** 2 )**(1/3), axis=1)
test['gy_t']  = test.apply(lambda x : (x['gy_x']**2 + x['gy_y'] **2 +  x['gy_z'] ** 2 )**(1/3), axis=1)

In [30]:
# https://dacon.io/competitions/official/235689/codeshare/2375?page=1&dtype=recent&ptype=pub
# 간단한 LGBM으로 접근해보기

feature = ['acc_x', 'acc_y', 'acc_z', 'acc_t', 'gy_x', 'gy_y', 'gy_z', 'gy_t']

X_pivot_train = pd.pivot_table(data = train, # X_train의 데이터를 통해서
                               values = feature,  # id와 time을 제외한 피쳐를 대상으로
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',         # 합, 평균
                                          'median','min','max', # 중앙값 최소값, 최대값
                                          'std','var','skew'   # 베셀 보정 표본 표준편차, 비편향 편차 의 값을 구합니다.
                                         ]
                              )

X_pivot_test = pd.pivot_table(data = test, #
                               values = feature, 
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',        
                                          'median','min','max',
                                          'std','var','skew'
                                         ]
                              )

In [31]:
X_columns = [agg + '_' + column for agg,column in X_pivot_train.columns]

X_pivot_train.columns = X_columns
X_pivot_test.columns = X_columns

X_pivot_train = X_pivot_train.reset_index()
X_pivot_test = X_pivot_test.reset_index()


train_data = pd.merge(X_pivot_train, train_label.loc[:,['id','label']], on='id') # label_desc는 사용하지 않을 예정입니다.
train_data.label = train_data.label.astype('category')

train_data.shape

(3125, 66)

In [32]:
feature_acc = ['sum_acc_t', 'sum_acc_x', 'sum_acc_y', 'sum_acc_z']
feature_gy = ['sum_gy_t', 'sum_gy_x', 'sum_gy_y', 'sum_gy_z']

X_pivot_train['sum_acc'] = np.sum(X_pivot_train[feature_acc], axis=1)
X_pivot_train['mean_acc'] = np.mean(X_pivot_train[feature_acc], axis=1)
X_pivot_train['median_acc'] = np.median(X_pivot_train[feature_acc], axis=1)
X_pivot_train['min_acc'] = np.min(X_pivot_train[feature_acc], axis=1)
X_pivot_train['max_acc'] = np.max(X_pivot_train[feature_acc], axis=1)
X_pivot_train['std_acc'] = np.std(X_pivot_train[feature_acc], axis=1)
X_pivot_train['var_acc'] = np.var(X_pivot_train[feature_acc], axis=1)
X_pivot_train['skew_acc'] = skew(X_pivot_train[feature_acc], axis=1)

X_pivot_train['sum_gy'] = np.sum(X_pivot_train[feature_gy], axis=1)
X_pivot_train['mean_gy'] = np.mean(X_pivot_train[feature_gy], axis=1)
X_pivot_train['median_gy'] = np.median(X_pivot_train[feature_gy], axis=1)
X_pivot_train['min_gy'] = np.min(X_pivot_train[feature_gy], axis=1)
X_pivot_train['max_gy'] = np.max(X_pivot_train[feature_gy], axis=1)
X_pivot_train['std_gy'] = np.std(X_pivot_train[feature_gy], axis=1)
X_pivot_train['var_gy'] = np.var(X_pivot_train[feature_gy], axis=1)
X_pivot_train['skew_gy'] = skew(X_pivot_train[feature_gy], axis=1)

X_pivot_test['sum_acc'] = np.sum(X_pivot_test[feature_acc], axis=1)
X_pivot_test['mean_acc'] = np.mean(X_pivot_test[feature_acc], axis=1)
X_pivot_test['median_acc'] = np.median(X_pivot_test[feature_acc], axis=1)
X_pivot_test['min_acc'] = np.min(X_pivot_test[feature_acc], axis=1)
X_pivot_test['max_acc'] = np.max(X_pivot_test[feature_acc], axis=1)
X_pivot_test['std_acc'] = np.std(X_pivot_test[feature_acc], axis=1)
X_pivot_test['var_acc'] = np.var(X_pivot_test[feature_acc], axis=1)
X_pivot_test['skew_acc'] = skew(X_pivot_test[feature_acc], axis=1)

X_pivot_test['sum_gy'] = np.sum(X_pivot_test[feature_gy], axis=1)
X_pivot_test['mean_gy'] = np.mean(X_pivot_test[feature_gy], axis=1)
X_pivot_test['median_gy'] = np.median(X_pivot_test[feature_gy], axis=1)
X_pivot_test['min_gy'] = np.min(X_pivot_test[feature_gy], axis=1)
X_pivot_test['max_gy'] = np.max(X_pivot_test[feature_gy], axis=1)
X_pivot_test['std_gy'] = np.std(X_pivot_test[feature_gy], axis=1)
X_pivot_test['var_gy'] = np.var(X_pivot_test[feature_gy], axis=1)
X_pivot_test['skew_gy'] = skew(X_pivot_test[feature_gy], axis=1)

X_pivot_train.shape, X_pivot_test.shape

((3125, 81), (782, 81))

In [33]:
from sklearn.preprocessing import RobustScaler
transformer_pivot = RobustScaler().fit(X_pivot_train.drop('id', axis=1))
pivot_train = transformer_pivot.transform(X_pivot_train.drop('id', axis=1))
pivot_test = transformer_pivot.transform(X_pivot_test.drop('id', axis=1))

In [42]:
# 푸리에 변환

def ft_trans(name,train,test):
    def train_test(check,num_col):

        
        if check =='train':
            df_checking=train.copy()
            train_datas = np.zeros((len(df_checking.id.unique()),304))
            
        elif check =='test':
            df_checking=test.copy()
            train_datas = np.zeros((len(df_checking.id.unique()),304))
                       

        for i,num in enumerate(tqdm(df_checking.id.unique())):

            tt = df_checking.loc[df_checking.id==num][name] -df_checking.loc[df_checking.id==num][name].mean()
            fmax = 50      # sampling frequency 1000 Hz
            dt = 1/fmax      # sampling period
            N  = 600      # length of signal

            t  = np.arange(0,N)*dt   # time = [0, dt, ..., (N-1)*dt]
            x = tt.values
            df = fmax/N   # df = 1/N = fmax/N
            f = np.arange(0,N)*df     #   frq = [0, df, ..., (N-1)*df]
            xf = np.fft.fft(x)*dt
            tq_index=f[0:int(N/2+1)]
            tq_abs= np.abs(xf[0:int(N/2+1)])

            results = pd.DataFrame(tq_abs,tq_index).reset_index().rename(columns={'index':'hz',0:'abs_value'})
            
            ar0 = np.array([num])
            ar1 =results.abs_value.values
            ar2 = np.array([skew(results.abs_value),kurtosis(results.abs_value, fisher=True)])
            return_value = np.concatenate([ar0,ar1 ,ar2])    
            train_datas[i] = return_value

        return train_datas

    
    col_ft = ['_'+str(x) for x in range(304)]
    
    num_col = len(col_ft)
    train_datas = train_test('train',num_col)
    test_datas = train_test('test',num_col)
    
    col_ft_F = ['id']+[name+"_"+x for x in col_ft[1:]]        
    train_df = pd.DataFrame(train_datas,columns= col_ft_F)
    test_df = pd.DataFrame(test_datas,columns= col_ft_F)
    
    train_df.id = train_df.id.astype('int')
    test_df.id = test_df.id.astype('int')
    
    
    return train_df ,test_df

# https://dacon.io/competitions/official/235689/codeshare/2374?page=1&dtype=recent&ptype=pub

In [47]:
train_fft,test_fft = ft_trans('acc_t',train,test)

train_fft['sum_acc'] = np.sum(train_fft, axis=1)
train_fft['mean_acc'] = np.mean(train_fft, axis=1)
train_fft['median_acc'] = np.median(train_fft, axis=1)
train_fft['min_acc'] = np.min(train_fft, axis=1)
train_fft['max_acc'] = np.max(train_fft, axis=1)
train_fft['std_acc'] = np.std(train_fft, axis=1)
train_fft['var_acc'] = np.var(train_fft, axis=1)
train_fft['skew_acc'] = skew(train_fft, axis=1)

test_fft['sum_acc'] = np.sum(test_fft, axis=1)
test_fft['mean_acc'] = np.mean(test_fft, axis=1)
test_fft['median_acc'] = np.median(test_fft, axis=1)
test_fft['min_acc'] = np.min(test_fft, axis=1)
test_fft['max_acc'] = np.max(test_fft, axis=1)
test_fft['std_acc'] = np.std(test_fft, axis=1)
test_fft['var_acc'] = np.var(test_fft, axis=1)
test_fft['skew_acc'] = skew(test_fft, axis=1)

train_fft.shape, test_fft.shape

100%|██████████| 782/782 [00:03<00:00, 232.63it/s]


((3125, 312), (782, 312))

In [62]:
# 모델 1번: xgboost

def build_xgboost(split_num, train, target, test, rnd):
    
    params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 10,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':61,
                'tree_method' : 'gpu_hist'
                }
    
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))
    
    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        d_train = xgb.DMatrix(X, y)
        d_valid = xgb.DMatrix(valid_x, valid_y)
        d_temp = xgb.DMatrix(valid_x)
        d_test = xgb.DMatrix(test)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        
        #run traning
        model = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=100)

        # save feat
        train_pred[val_idx] = model.predict(d_temp)
        test_pred += model.predict(d_test)/split_num
        
        # release
        del model
        gc.collect()
        print('  ==============================================================  ')
        
    return train_pred, test_pred

xgb_train_fft, xgb_test_fft = build_xgboost(5, np.array(train_fft), np.array(train_data.label), np.array(test_fft), 1)
xgb_train, xgb_test = build_xgboost(5, pivot_train, np.array(train_data.label), pivot_test, 1)

[0]	train-mlogloss:3.602	valid-mlogloss:3.65173
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.230882	valid-mlogloss:1.68981
[200]	train-mlogloss:0.05661	valid-mlogloss:1.63105
Stopping. Best iteration:
[243]	train-mlogloss:0.040211	valid-mlogloss:1.62812

[0]	train-mlogloss:3.61236	valid-mlogloss:3.63774
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.227659	valid-mlogloss:1.71956
[200]	train-mlogloss:0.055569	valid-mlogloss:1.67959
Stopping. Best iteration:
[179]	train-mlogloss:0.068766	valid-mlogloss:1.6777

[0]	train-mlogloss:3.60843	valid-mlogloss:3.65463
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[100]	train-mlogloss:0.

In [63]:
# 모델 2번: catboost

def build_catboost(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))

    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):
        
        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]
        
        model = cb.CatBoostClassifier(iterations=30000,
                                      learning_rate=0.01,
                                      l2_leaf_reg=3.5,
                                      depth=6,
                                      loss_function= 'MultiClass',
                                      eval_metric='MultiClass',
                                      use_best_model=True,
                                      random_seed=42,
                                      verbose=500,
                                      task_type="GPU")
        
        model.fit(X, y,
                  eval_set=(valid_x, valid_y),
                  early_stopping_rounds=50)
        
        # save feat
        train_pred[val_idx] = model.predict_proba(valid_x)
        test_pred += model.predict_proba(test)/(split_num)
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

catboost_train_fft, catboost_test_fft = build_catboost(5, np.array(train_fft), train_data.label, np.array(test_fft), 1)
catboost_train, catboost_test = build_catboost(5, np.array(X_pivot_train.drop('id', axis=1)), train_data.label, np.array(X_pivot_test.drop('id', axis=1)), 1)

0:	learn: 3.9416156	test: 3.9467965	best: 3.9467965 (0)	total: 86.1ms	remaining: 43m 2s
500:	learn: 1.4849907	test: 1.7883893	best: 1.7883893 (500)	total: 43.1s	remaining: 42m 20s
1000:	learn: 1.1278936	test: 1.6272023	best: 1.6272023 (1000)	total: 1m 24s	remaining: 40m 53s
1500:	learn: 0.8965914	test: 1.5483399	best: 1.5483399 (1500)	total: 2m 5s	remaining: 39m 36s
2000:	learn: 0.7218663	test: 1.5030455	best: 1.5030455 (2000)	total: 2m 44s	remaining: 38m 28s
2500:	learn: 0.5898925	test: 1.4716727	best: 1.4716727 (2500)	total: 3m 24s	remaining: 37m 28s
3000:	learn: 0.4876501	test: 1.4512870	best: 1.4512552 (2999)	total: 4m 3s	remaining: 36m 34s
3500:	learn: 0.4058656	test: 1.4360523	best: 1.4360241 (3498)	total: 4m 43s	remaining: 35m 43s
4000:	learn: 0.3407309	test: 1.4253670	best: 1.4252735 (3996)	total: 5m 22s	remaining: 34m 54s
4500:	learn: 0.2888949	test: 1.4200766	best: 1.4200766 (4500)	total: 6m 1s	remaining: 34m 9s
bestTest = 1.417306055
bestIteration = 4793
Shrink model to firs

## Ensemble

In [64]:
trans_onehot = np.argmax(transformer_train, axis=1).reshape(-1,1)
xgb_onehot = np.argmax(xgb_train_fft, axis=1).reshape(-1,1)
xgb_fft_onehot = np.argmax(xgb_train, axis=1).reshape(-1,1)
catboost_onehot = np.argmax(catboost_train, axis=1).reshape(-1,1)
catboost_fft_onehot = np.argmax(catboost_train_fft, axis=1).reshape(-1,1)

trans_onehot_test = np.argmax(transformer_test, axis=1).reshape(-1,1)
xgb_onehot_test = np.argmax(xgb_test, axis=1).reshape(-1,1)
xgb_fft_onehot_test = np.argmax(xgb_test_fft, axis=1).reshape(-1,1)
catboost_onehot_test = np.argmax(catboost_test, axis=1).reshape(-1,1)
catboost_fft_onehot_test = np.argmax(catboost_test_fft, axis=1).reshape(-1,1)

train_final = np.hstack([xgb_onehot,
                         xgb_fft_onehot,
                         catboost_onehot,
                         trans_onehot,
                         catboost_fft_onehot])

test_final = np.hstack([xgb_onehot_test,
                        xgb_fft_onehot_test,
                        catboost_onehot_test,
                        trans_onehot_test,
                        catboost_fft_onehot_test])

concat_train = np.concatenate((pivot_train, train_final), axis=1)
concat_test = np.concatenate((pivot_test, test_final), axis=1)

print(concat_train.shape)
print(concat_test.shape)

# print(train_final.shape)
# print(test_final.shape)

# https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221343373342&proxyReferer=https:%2F%2Fwww.google.com%2F

(3125, 85)
(782, 85)


In [65]:
# 모델 2번: catboost

def ensemble(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob
    test_pred = np.zeros((test.shape[0], 61))

    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):
        
        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]
        
        model = cb.CatBoostClassifier(iterations=30000,
                                      learning_rate=0.01,
                                      l2_leaf_reg=3.5,
                                      depth=6,
                                      loss_function= 'MultiClass',
                                      eval_metric='MultiClass',
                                      use_best_model=True,
                                      random_seed=42,
                                      verbose=500,
                                      task_type="GPU")
        
        model.fit(X, y,
                  eval_set=(valid_x, valid_y),
                  early_stopping_rounds=50)
        
        # save feat
        test_pred += model.predict_proba(test)/(split_num)
        
        # release
        del model
        gc.collect()
        print('------------------')
    
    sample_submssion = pd.read_csv(path + 'sample_submission.csv')
    sample_submssion.iloc[:,1:] = test_pred
    sample_submssion.to_csv("ensemble_with_fft.csv", index = False)

ensemble(5, concat_train, train_data.label, concat_test, 1)

0:	learn: 3.9630559	test: 3.9666457	best: 3.9666457 (0)	total: 79.1ms	remaining: 39m 33s
500:	learn: 0.8672576	test: 1.0596146	best: 1.0596146 (500)	total: 19.7s	remaining: 19m 19s
1000:	learn: 0.5606591	test: 0.8546825	best: 0.8546825 (1000)	total: 38.8s	remaining: 18m 45s
1500:	learn: 0.4162530	test: 0.7739517	best: 0.7739517 (1500)	total: 57.8s	remaining: 18m 18s
2000:	learn: 0.3240402	test: 0.7290292	best: 0.7290292 (2000)	total: 1m 16s	remaining: 17m 49s
2500:	learn: 0.2596616	test: 0.6999210	best: 0.6999210 (2500)	total: 1m 34s	remaining: 17m 23s
3000:	learn: 0.2118759	test: 0.6810342	best: 0.6810342 (3000)	total: 1m 53s	remaining: 16m 59s
3500:	learn: 0.1757229	test: 0.6681668	best: 0.6681668 (3500)	total: 2m 11s	remaining: 16m 34s
4000:	learn: 0.1475955	test: 0.6586176	best: 0.6586176 (4000)	total: 2m 29s	remaining: 16m 12s
4500:	learn: 0.1251942	test: 0.6514407	best: 0.6514371 (4499)	total: 2m 48s	remaining: 15m 53s
5000:	learn: 0.1079565	test: 0.6456812	best: 0.6456812 (5000)

In [ ]:
# https://www.kaggle.com/gogo827jz/jane-street-ffill-transformer-baseline
# https://wikidocs.net/31379
# https://www.tensorflow.org/tutorials/text/transformer